# Reinforcement Learning from Human Feedback

In [2]:
!pip install trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:000:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires nump

In [3]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments
from trl import RewardTrainer, RewardConfig

2025-06-19 11:59:23.488725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750334363.697438      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750334363.758456      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_data = load_dataset("Vezora/Code-Preference-Pairs", split="train")
train_data.to_pandas().head()

README.md:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

Code-Preference-Pairs.jsonl:   0%|          | 0.00/214M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/54024 [00:00<?, ? examples/s]

,instruction,input,accepted,rejected,ID
0,You are an AI-Coding assistant. User will you ...,Create a nested loop to print every combinatio...,Here is an example of a nested loop in Python ...,Here is an example of a nested loop in Python ...,1
1,You are an AI-Coding assistant. User will you ...,Write a function to find the number of distinc...,The given problem can be solved by iterating t...,The given problem can be solved by iterating t...,2
2,You are an AI-Coding assistant. User will you ...,Write a function that checks if a given number...,Here is an implementation of the function in P...,Here is an implementation of the function in P...,3
3,You are an AI-Coding assistant. User will you ...,Write code to find the sum of all prime number...,"To solve this problem, we can create a functio...","To solve this problem, we can create a functio...",4
4,You are an AI-Coding assistant. User will you ...,Create an array of length N (where N is a posi...,Here is one possible implementation of the sol...,Here is one possible implementation of the sol...,5


In [5]:
model_name = "google-t5/t5-small"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

## Reward Model

In [ ]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation":True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = examples["input"] + "\n" + examples["accepted"]
    prompt_plus_rejected_response = examples["input"] + "\n" + examples["rejected"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0],
        "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0],
        "attention_mask_rejected": tokens_rejected["attention_mask"][0],
        "return_loss": True,
    }

formatted_dataset = train_data.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

In [ ]:
training_args = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="steps",
    logging_steps=1,
    num_train_epochs=1,
    report_to=None,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    remove_unused_columns=False,     # Already handled in preprocessing
    save_strategy="epoch",
    max_length=512,
)

In [ ]:
trainer = RewardTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
)

trainer.train()

In [ ]:
!huggingface-cli login

In [ ]:
# model.push_to_hub("Kira-Floris/RM-t5-small-code-preference-dataset")
# tokenizer.push_to_hub("Kira-Floris/RM-t5-small-code-preference-dataset")

## Reinforcement Learning

In [6]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM

In [7]:
from transformers import pipeline, AutoModelForSequenceClassification

# reward_model = pipeline("text-classification", model="Kira-Floris/RM-t5-small-code-preference-dataset")
reward_model = AutoModelForSequenceClassification.from_pretrained("Kira-Floris/RM-t5-small-code-preference-dataset")
reward_tokenizer = AutoTokenizer.from_pretrained("Kira-Floris/RM-t5-small-code-preference-dataset")

reward_tokenizer.padding_side = "left"
if reward_tokenizer.pad_token is None:
    reward_tokenizer.pad_token = reward_tokenizer.eos_token

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/243M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [8]:
rl_model_name = "google-t5/t5-small"
rl_model_name = "gpt2"

# rl_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(rl_model_name)
rl_model = AutoModelForCausalLM.from_pretrained(rl_model_name)
# from copy import deepcopy
rl_model_ref = create_reference_model(rl_model)
# rl_model_ref = deepcopy(rl_model)
# for param in rl_model_ref.parameters():
#     param.requires_grad = False

rl_tokenizer = AutoTokenizer.from_pretrained(rl_model_name)
rl_tokenizer.pad_token = tokenizer.eos_token

def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["input"], padding=True, truncation=True, max_length=512)
    return sample

rl_dataset = train_data.map(tokenize, batched=False, remove_columns=train_data.column_names)
rl_dataset = rl_dataset.train_test_split(test_size=0.1)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/54024 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

rl_datacollator = DataCollatorWithPadding(tokenizer=rl_tokenizer, return_tensors='pt')

rl_trainloader = DataLoader(
    rl_dataset["train"], shuffle=True, batch_size=4, collate_fn=rl_datacollator
)
rl_testloader = DataLoader(
    rl_dataset["test"], shuffle=False, batch_size=4, collate_fn=rl_datacollator
)

In [ ]:
ppo_config = PPOConfig(
    learning_rate=1e-5,
    batch_size=1,
    mini_batch_size=1
)

ppo_trainer = PPOTrainer(
    args=ppo_config,
    model=rl_model,
    value_model=rl_model,
    reward_model=reward_model,
    ref_model=rl_model_ref,
    processing_class=tokenizer,
    train_dataset=rl_dataset["train"],
    eval_dataset=rl_dataset["test"],
    data_collator=rl_datacollator,
)

In [ ]:
type(rl_dataset["train"])

In [10]:
import torch
import torch.nn.functional as F
from torch.optim import AdamW

beta = 0.1
optimizer = AdamW(rl_model.parameters(), lr=1e-5)

In [ ]:
# for epoch in range(1):
#     for batch in rl_trainloader:
#         inputs = {k:v for k,v in batch.items()}
#         # print("inputs",rl_tokenizer.batch_decode(inputs["input_ids"], skip_special_token=True, clean_up_tokenization_spaces=True)[0])
        
#         outputs = rl_model.generate(**inputs, max_new_tokens=50, output_scores=True, return_dict_in_generate=True, output_logits=True)
#         rl_probabilities = outputs.logits
#         # print(rl_probabilities)
#         rl_outputs = rl_tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True, clean_up_tokenization_spaces=True)
#         print("Shape of RL Logits;", len(rl_probabilities), rl_probabilities[0].shape)

#         # reference model
#         ref_outputs = rl_model_ref.generate(**inputs, max_new_tokens=50, output_scores=True, return_dict_in_generate=True, output_logits=True)
#         ref_probabilities = ref_outputs.logits
#         print("Shape of Ref Logits:", len(ref_probabilities), ref_probabilities[0].shape)

#         # reward model
#         rl_tokenized_inputs = reward_tokenizer(rl_outputs, padding=True, truncation=True, return_tensors="pt")
#         reward_outputs = reward_model(**rl_tokenized_inputs)
#         reward_probabilities = reward_outputs.logits
#         print("Shape of Reward Logits:",reward_probabilities.shape)

        
#         break

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

policy = rl_model.to(device)
policy_ref = rl_model_ref.to(device)
reward_model = reward_model.to(device)

In [ ]:
for epoch in range(3):
    losses = 0.0
    for batch in rl_trainloader:
        inputs = {k: v.to(device) for k, v in batch.items()}

        gen = policy.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True, top_k=50, top_p=0.9, temperature=1.0,
            output_scores=True, return_dict_in_generate=True, output_logits=True,
            pad_token_id=rl_tokenizer.eos_token_id,
        )
        ref_gen = policy_ref.generate(
            **inputs,
            max_new_tokens=50,
            output_scores=True, return_dict_in_generate=True, output_logits=True,
            pad_token_id=rl_tokenizer.eos_token_id,
        )

        sequences = gen.sequences
        B = sequences.size(0)
        prompt_len = inputs["input_ids"].size(1)
        T = sequences.size(1) - prompt_len

        # Extract token log-probs
        policy_lps = [F.log_softmax(l, dim=-1) for l in gen.logits]
        ref_lps = [F.log_softmax(l, dim=-1) for l in ref_gen.logits]

        gen_ids = sequences[:, prompt_len:]  # shape (B, T)

        policy_token_lps = torch.stack([
            policy_lps[t].gather(1, gen_ids[:, t].unsqueeze(1)).squeeze(1)
            for t in range(T)
        ], dim=1)  # shape (B, T)

        ref_token_lps = torch.stack([
            ref_lps[t].gather(1, gen_ids[:, t].unsqueeze(1)).squeeze(1)
            for t in range(T)
        ], dim=1)

        logp_policy = policy_token_lps.sum(dim=1)  # shape (B,)
        logp_ref = ref_token_lps.sum(dim=1)       # shape (B,)

        # Score responses using the reward model
        texts = rl_tokenizer.batch_decode(
            sequences, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        reward_inputs = reward_tokenizer(
            texts, padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        reward_logits = reward_model(**reward_inputs).logits  # e.g. shape (B, 2)

        # Extract a single scalar reward per example
        if reward_logits.ndim == 2:
            rewards = reward_logits[:, 1]  # shape (B,)
        else:
            rewards = reward_logits.view(-1)

        # Debug: confirm shapes
        # print(".", logp_policy.shape, logp_ref.shape, rewards.shape)  # all should be (B,)

        # Compute KL‑penalized RL loss
        loss = -torch.mean(rewards - beta * (logp_policy - logp_ref))
        losses += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch = {epoch} -- loss = {losses.item():.4f}")
        

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [ ]:
# max_ppo_steps = 10
# output_min_length = 16
# output_max_length = 128
# output_length_sampler = lambda: random.randint(output_min_length, output_max_length)

# generation_kwards= {
#     "min_length":5,
#     "top_k": 0.0,
#     "top_p":1.0,
#     "do_sample":True
# }

# reward_kwargs = {
#     "batch_size":16
# }

# for step, batch in enumerate(ppo_trainer.dataloader):
#     if step >= max_ppo_steps:
#         break

#     prompt_tensors = batch["input_ids"]

#     summary_tensors = []

#     for prompt_tensor in prompt_tensors:
#         max_new_tokens = output_length_sampler()
#         generation_kwards["max_new_tokens"] = max_new_tokens

#         summary = rl_model.generate(prompt_tensor.unsqueeze(0))
#         summary_tensors.append(summary.squeeze()[-max_new_tokens:])

#     batch["response"] = [rl_tokenizer.decode(r.squeeze()) for r in summary_tensors]

#     query_response_pairs = [q+r for q, r in zip(rl_tokenizer.decode(prompt_tensors[0]), batch["response"])]
#     # print(query_response_pairs, type(query_response_pairs[0]))
#     # break
#     query_tokenized = reward_tokenizer(query_response_pairs, padding=True, truncation=True, return_tensors="pt")
#     query_tokenized = {k:v.to(reward_model.device) for k, v in query_tokenized.items()}
#     reward_logits = reward_model(query_tokenized["input_ids"])

#     print(reward_logits)
#     break


#     stats = ppo_trainer.step(prompt_tensors, summary_tensors, rewards)
#     ppo_trainer.log_stats(stats, batch, rewards)